# Artificial and Computational Intelligence Assignment 1

## Problem solving by Informed Search (A* Algorithm)

Active contributors in this assignment:
1. KANURU NAGESWARA RAO (BITS ID: **2021sc04876**)
2. ARUN MATHEW (BITS ID: **2021sc04982**)
3. THAKKAR PRACHI CHETAN CHETNA (BITS ID: **2021sc04875**)

### 1.	Define the environment in the following block

<h4>PEAS decription of the given problem statement.</h4>

<p><h6><u>PSA Agent(Problem Solving Agent) </u></h6></p>

<p><b>Agent</b> : We have two agents R1 and G1 that are neither competing nor collaborating with each other. 
<p><b>Performance</b> : Cost required to reach the goal ( Reaching the goal with the lowest cost is the expectation )
<p><b>Environment</b> : The environment consists of 2 scenarios - Scenario 1 (Grid 1) and Scenario 2 (Grid 2). Both the scenarios have Green and Red coloured rooms arranged in different patterns. 
<p><b>Sensors</b> : Colour identification program for identifying room colour of the agent and neighbour nodes - Green and Red colored rooms. 
<p><b>Actuators</b> : Program to move the agents in one of the 4 available directions - North, South, East, West. Display of marking for agents visit.




#### Environment Definition

In [1]:
# Import libraries
import numpy as np
import random
import heapq
import time
import numpy as np
import pandas as pd
from IPython.display import display, HTML



In [2]:
#Code Block : Set Initial State (Must handle dynamic inputs)

# Define the agents. 
# There are 2 distinct agents R1 and G1, both trying to navigate through grid searching for the goal node. 
agents = ['R1', 'G1']

# Available colours in the grid
# The grid has 2 colours. Green and Red. 
colours = {
    0:'Green',
    1:'Red'
}

# Obstacle room for each agent
# R1 has additional penalty
agent_obstacle = {
    'R1':'Green',
    'G1': 'Red'
}

<b> Create the grid environments for search

Both the environments are represented using 6*6 matrices containing values 0 and 1. 
<ul>
   <li> 0 > indicates green
   <li> 1 > indicates red
</ul>

In [3]:

# Create the Grid for scenario 1
def create_grid1():
    grid = np.zeros([6,6],dtype=int)
    grid[0,0] = 1
    grid[0,4] = 1
    grid[1,3] = 1
    grid[2,2] = 1
    grid[3,1] = 1
    grid[4,0] = 1
    grid[3,5] = 1
    grid[4,4] = 1
    grid[5,3] = 1
    return grid

grid1 = create_grid1()

# Create the grid for scenario 2
def build_checkerboard(w, h) :
    re = np.r_[ w*[1,0] ]              # even-numbered rows
    ro = np.r_[ w*[0,1] ]              # odd-numbered rows
    return np.row_stack(h*(re, ro))

def create_grid2():
    return build_checkerboard(3,3)

grid2 = create_grid2()

print('-------Grid 1 ------')
print(grid1)
print()
print('-------Grid 2 ------')
print(grid2)


-------Grid 1 ------
[[1 0 0 0 1 0]
 [0 0 0 1 0 0]
 [0 0 1 0 0 0]
 [0 1 0 0 0 1]
 [1 0 0 0 1 0]
 [0 0 0 1 0 0]]

-------Grid 2 ------
[[1 0 1 0 1 0]
 [0 1 0 1 0 1]
 [1 0 1 0 1 0]
 [0 1 0 1 0 1]
 [1 0 1 0 1 0]
 [0 1 0 1 0 1]]


#### Utility Methods

In [4]:
#Code Block : Set the matrix for transition & cost (as relevant for the given problem)

# Fetch the colour of the node/room from the grid
def get_colour(node, grid):
    return grid[node[0], node[1]]

# Check whether the node is having a colour different from the goal node
def is_wrong_coloured_room(node, goal, grid):
    return get_colour(node, grid) != get_colour(goal, grid)

def is_obstacle_room(node, grid, agent):
    color_code = get_colour(node, grid)
    return agent_obstacle[agent] == colours[color_code]



# Calculate the heuristic cost estimate
# The heuristic cost is manhattan distance +/- color penalty
# If the node and the goal are in same coloured nodes, then a penalty of (-5) will be added and (+5) otherwise. 
def heuristic_cost_estimate(node, goal, grid):
    # Calculate the Manhattan distance between the node and the goal
    manhattan_distance = abs(node[0] - goal[0]) + abs(node[1] - goal[1])
    
    #Calculate the color penalty
    if is_wrong_coloured_room(node, goal, grid): # node and goal are not in the same-colored room
        color_penalty = 5
    else:
        color_penalty = -5

    return manhattan_distance + color_penalty

# Get the cost of the room.
# If the agent is moving to an obstacle room, the cost of +10 will be penalty. Else, (-10) will be the penalty (reward). 
def get_room_cost(current, neighbor, grid, agent):
    # enable the below section to prevent further reduction of room cost if the agent is not moving to a 
    # different coloured room
    #if get_colour(current, grid) == get_colour(neighbor, grid):
    #    cost = 0
    
    if is_obstacle_room(neighbor, grid, agent):
        cost = 10  # Incur a penalty for entering the wrong-colored room
    else:
        cost = -10  # Get a bonus for entering the correct-colored room
    return cost


#### Utility Methods : Find the neighbours of a node

In [5]:
# Given a node, find the neighbours of that node.
def find_neighbours(node, grid):
    possible_movements = [(0,1),(1,0), (0,-1),(-1,0)]
    neighbours = []
    g_shape = grid.shape
    for i in possible_movements:
        step = (node[0]+i[0], node[1]+i[1])
        if step[0] > -1 and step[1] > -1 and step[0]<g_shape[0] and step[1]<g_shape[1]:
            neighbours.append(step)
    return neighbours

def reconstruct_path(cameFrom, current):
    total_path = [current]
    while current in cameFrom:
        current = cameFrom[current]
        total_path.append(current)
    return total_path


In [6]:
# Create a random start point. 
def create_start_point(grid):
    grid_shape = grid.shape
    x = random.randint(0,grid_shape[0]-1)
    y = random.randint(0,grid_shape[1]-1)
    return (x,y)


# Print the detailed logs generated by the search algorithm (A*)
def log_path(logs, show_neighbors):
    print('\033[91m', 'Printing navigation logs')
    main_logs = []
    for node in logs:
        #print('\033[92m', 'Current Node', node['node'], 'Cost', node['gcost'], 'Color', node['color'], 'Is Goal', node['is_goal'])
        row_log = [str(node['node']), node['gcost'], node['hcost'], node['fcost'],node['color'],node['is_goal']]
        main_logs.append(row_log)
        if show_neighbors and node['node'] != goal:
            neighbors = node['neighbors']
            for neighbor in neighbors:
                if neighbore_node['opening'] == 1: 
                    print('\033[94m','         ','node:',neighbor['node'], 'cost:', neighbor['cost'], 'hcost:',neighbore_node['hscore'], 'gcost:',neighbore_node['gscore'], 'color:',neighbor['color'])
    
    print('\033[90m')
    df = pd.DataFrame(main_logs, columns=['Node', 'Step Cost', 'Heuristic Cost', 'Total Cost', 'Node Colour', 'Goal Node?'])
    display(df)
    

Define the goal nodes

In [7]:
# Goal node of grid 1
grid1_goal = (2,3)

#Goal node of grid 2
grid2_goal = (2,2)

### 2.	Solution using A* Algorithm

In [8]:
#Code Block : Function for algorithm 1 implementation
failure = (-1,-1)
logs = []



def a_star(start, goal, grid, agent):
    space_used = 0
    
    # Total space used by the program
    # Each neighbour node opened by the algorithm adds to the space complexity
    

    # Total time taken by the program
    # Time complexity is the time in milliseconds taken to complete the program. 
    # If the algorithm has to open more number of nodes, time complexity will increase.
    time_taken = 0
    
    if not (agent == 'R1' or agent == 'G1'):
        print('Unknown agent. Agent should be \'R1\' or \'G1\'')
    
    # initialize the start time as current system time
    start_time = time.time()
    
    #reinitialize logs
    logs.clear()
    print('Starting node ', start, 'Goal Node ', goal)

    #The set of nodes already evaluated
    closedSet = set() 
    
    #The set of currently discovered nodes that are not evaluated yet.
    #Initially, only the start node is known.
    openSet = set()    
    openSet.add(start)
    space_used = space_used + 1
    
    #For each node, which node it can most efficiently be reached from.
    #If a node can be reached from many nodes, cameFrom will eventually contain the
    #most efficient previous step.
    cameFrom = {}

    #For each node, the cost of getting from the start node to that node.
    gScore = {start:0}
    shScore =heuristic_cost_estimate(start, goal, grid)
    hScore = {start:shScore}
    #The cost of going from start to start is zero.
    fscore = {start:shScore}
    oheap = []
    heapq.heappush(oheap, (fscore[start], start))
    while oheap: # is not empty
        # current -> the node in openSet with the lowest fScore[] value
        cost, current = heapq.heappop(oheap)
        col = get_colour(current, grid)
        log_data = {'node':current, 'gcost':gScore[current], 'hcost':hScore[current], 'fcost':cost,  'color':colours[col], 'is_goal':0}
        if current == goal:
            log_data['is_goal'] = 1
            print('Found goal ', goal)
            logs.append(log_data)
            final_path = reconstruct_path(cameFrom, current)
            
            # Calculate the end time as current system time just before returning the final path
            end_time = time.time()
            time_taken = end_time - start_time
            return final_path, space_used, time_taken
        
        #oheap.remove(current)
        closedSet.add(current)
        neighbors = find_neighbours(current, grid)
        neighbor_log = []
        for neighbor in neighbors:
            if neighbor in closedSet:
                continue        # Ignore the neighbor which is already evaluated.
    
            # Calculate the cost of transitioning to the neighbor
            cost = 1  # Base path cost
            cost += get_room_cost(current, neighbor, grid, agent)
            tentative_gScore = gScore[current] + cost   
            #logging data
            neighbore_node = {'node':neighbor, 'color':colours[col]}
            
            if neighbor not in openSet  : # Discover a new node
                openSet.add(neighbor)
                space_used = space_used + 1
            elif tentative_gScore >= gScore[neighbor]:
                #logging data
                neighbore_node['opening'] = 0
                continue        # This is not a better path.
            #logging data
            neighbore_node['opening'] = 1
            # This path is the best until now. Record it!
            cameFrom[neighbor] = current
            gScore[neighbor] = tentative_gScore
            hcost =  heuristic_cost_estimate(neighbor, goal, grid)
            hScore[neighbor] = hcost
            fscore[neighbor] = gScore[neighbor] + hcost
            heapq.heappush(oheap, (fscore[neighbor], neighbor))
            
            #data for logging
            neighbore_node['gscore'] = gScore[neighbor]
            neighbore_node['hscore'] = hcost
            neighbore_node['cost'] = fscore[neighbor]
            neighbor_log.append(neighbore_node)   
        
        log_data['neighbors'] = neighbor_log
        logs.append(log_data)
    
    # Calculate the end time if the program cannot find the path to the goal node
    end_time = time.time()
    time_taken = end_time - start_time 
    return failure, space_used, time_taken

### DYNAMIC INPUT

IMPORTANT : Dynamic Input must be got in this section. Display the possible states to choose from:
This is applicable for all the relevent problems as mentioned in the question. 

The program allows user to provide the start node as any arbitrary node in the grid.  Start point need to be specified as a valid tuple. 
The start point given will be passed to both the agents and both scenarios to trace the best path. 
Provide a valid tuple as the start node. Eg: (1,1) or specify value 'random' to generate a random start point

In [9]:
#Code Block : Function & call to get inputs (start/end state)

def is_in_range(start_eval):
    x = start_eval[0]
    y = start_eval[1]
    gshape = grid1.shape
    return type(x)==int and type(y)==int and x > -1 and x < gshape[0] and y > -1 and y < gshape[1]

    
print('Hint -> You can enter \'random\' to generate a start node dynamically \
or provide the start node in the format (x,y). The start node should be between (0,0) and (5,5)')

start = None
counter = 0
max_retry = 5

while start is None and counter < max_retry:
    dy_input = input('Enter your start node:  ')
    counter = counter + 1
    if dy_input.strip() == '':
        continue
    elif dy_input.lower() == 'random':
        print('Generating a random start point')
        start = create_start_point(grid1);
        print('Generated ',start,' as random start point')
    else:
        try:
            start_eval = eval(dy_input)
            if type(start_eval) is not tuple:
                continue
            else:
                if is_in_range(start_eval):
                    start = start_eval
                    print('Using ',start,'as start node.')
                else:
                    print('Start node is not valid')
        except Exception as e:
            print('Could not parse input value',e)

if counter >= max_retry:
    print('Could not create the start node.')
    print('Generating random start point')
    start = create_start_point(grid1)
    print('Generated ',start,' as random start point')



Hint -> You can enter 'random' to generate a start node dynamically or provide the start node in the format (x,y). The start node should be between (0,0) and (5,5)
Enter your start node:  (1,1)
Using  (1, 1) as start node.


### 4.	Calling the search algorithms

<b>Agent search for scenario 1 (Grid 1)<b>

Agent G1

In [10]:
#Invoke algorithm 1 (Should Print the solution, path, cost etc., (As mentioned in the problem))

# Starting agent path search for scenario1 using agent G1
agt_name = 'G1'
path, sc1_g1_space, sc1_g1_time = a_star(start, grid1_goal, grid1, agt_name)
path.reverse()
print('Final Path Taken by ',agt_name,' through Grid 1', '->'.join(str(x) for x in path))
print('---navigation logs--')
log_path(logs, False);

print('Space complexity of the program ',sc1_g1_space)
print('Time in seconds ',sc1_g1_time)



Starting node  (1, 1) Goal Node  (2, 3)
Found goal  (2, 3)
Final Path Taken by  G1  through Grid 1 (1, 1)->(1, 2)->(0, 2)->(0, 3)->(1, 3)->(2, 3)
---navigation logs--
 Printing navigation logs



,Node,Step Cost,Heuristic Cost,Total Cost,Node Colour,Goal Node?
0,"(1, 1)",0,-2,-2,Green,0
1,"(1, 2)",-9,-3,-12,Green,0
2,"(0, 2)",-18,-2,-20,Green,0
3,"(0, 3)",-27,-3,-30,Green,0
4,"(0, 1)",-27,-1,-28,Green,0
5,"(2, 1)",-9,-3,-12,Green,0
6,"(2, 0)",-18,-2,-20,Green,0
7,"(1, 0)",-27,-1,-28,Green,0
8,"(3, 0)",-27,-1,-28,Green,0
9,"(0, 1)",-27,-1,-10,Green,0


Space complexity of the program  17
Time in seconds  0.011993646621704102


Agent R1

In [11]:
# Starting agent path search for scenario1 using agent R1
agt_name = 'R1'
path, sc1_r1_space, sc1_r1_time = a_star(start, grid1_goal, grid1, agt_name)
path.reverse()
print('Final Path Taken by ',agt_name,' through Grid 1','->'.join(str(x) for x in path))
print('---navigation logs--')
log_path(logs, False);

print('Space complexity of the program ',sc1_r1_space)
print('Time in seconds ',sc1_r1_time)

Starting node  (1, 1) Goal Node  (2, 3)
Found goal  (2, 3)
Final Path Taken by  R1  through Grid 1 (1, 1)->(1, 2)->(1, 3)->(2, 3)
---navigation logs--
 Printing navigation logs



,Node,Step Cost,Heuristic Cost,Total Cost,Node Colour,Goal Node?
0,"(1, 1)",0,-2,-2,Green,0
1,"(1, 2)",11,-3,8,Green,0
2,"(1, 3)",2,6,8,Red,0
3,"(2, 1)",11,-3,8,Green,0
4,"(2, 2)",2,6,8,Red,0
5,"(2, 3)",13,-5,8,Green,1


Space complexity of the program  14
Time in seconds  0.0009989738464355469


<b>Agent search for scenario 2 (Grid 2)<b>

Agent G1

In [12]:
#Invoke algorithm 1 (Should Print the solution, path, cost etc., (As mentioned in the problem))

# Starting agent path search for scenario1 using agent G1
agt_name = 'G1'
path, sc2_g1_space, sc2_g1_time = a_star(start, grid2_goal, grid2, agt_name)
path.reverse()
print('Final Path Taken by ',agt_name,' through Grid 2','->'.join(str(x) for x in path))
print('---navigation logs--')
log_path(logs, False);

print('Space complexity of the program ',sc2_g1_space)
print('Time in seconds ',sc2_g1_time)

Starting node  (1, 1) Goal Node  (2, 2)
Found goal  (2, 2)
Final Path Taken by  G1  through Grid 2 (1, 1)->(1, 2)->(2, 2)
---navigation logs--
 Printing navigation logs



,Node,Step Cost,Heuristic Cost,Total Cost,Node Colour,Goal Node?
0,"(1, 1)",0,-3,-3,Red,0
1,"(1, 2)",-9,6,-3,Green,0
2,"(2, 1)",-9,6,-3,Green,0
3,"(2, 2)",2,-5,-3,Red,1


Space complexity of the program  10
Time in seconds  0.00099945068359375


Agent R1

In [13]:
# Starting agent path search for scenario1 using agent R1
agt_name = 'R1'
path, sc2_r1_space, sc2_r1_time = a_star(start, grid2_goal, grid2, agt_name)
path.reverse()
print('Final Path Taken by ',agt_name,' through Grid 2','->'.join(str(x) for x in path))
print('---navigation logs--')
log_path(logs, False);

print('Space complexity of the program ',sc2_r1_space)
print('Time in seconds ',sc2_r1_time)

Starting node  (1, 1) Goal Node  (2, 2)
Found goal  (2, 2)
Final Path Taken by  R1  through Grid 2 (1, 1)->(1, 2)->(2, 2)
---navigation logs--
 Printing navigation logs



,Node,Step Cost,Heuristic Cost,Total Cost,Node Colour,Goal Node?
0,"(1, 1)",0,-3,-3,Red,0
1,"(1, 2)",11,6,17,Green,0
2,"(2, 2)",2,-5,-3,Red,1


Space complexity of the program  8
Time in seconds  0.0010001659393310547


### 5.	Comparitive Analysis

In [14]:
#Code Block : Print the Time & Space complexity of algorithm 1 

data_space = [
    ['SC1', sc1_g1_space, sc1_r1_space, sc1_r1_space-sc1_g1_space ],
    ['SC2',sc2_g1_space, sc2_r1_space, sc2_r1_space-sc2_g1_space],
    ['DIFF',sc2_g1_space - sc1_g1_space,sc2_r1_space - sc1_r1_space,'']
]

space_df = pd.DataFrame(data_space, columns=['Scenario', 'Agent G1', 'Agent R1', 'Space Diff'])
theader = '<b>Space Complexity:</b>'+space_df.to_html(index=False)
HTML(theader)   

Scenario,Agent G1,Agent R1,Space Diff
SC1,17,14,-3
SC2,10,8,-2
DIFF,-7,-6,


In [15]:
#Code Block : Print the Time & Space complexity of algorithm 1 

data_time = [
    ['SC1', sc1_g1_time, sc1_r1_time, sc1_r1_time-sc1_g1_time ],
    ['SC2',sc2_g1_time, sc2_r1_space, sc2_r1_space-sc2_g1_time],
    ['DIFF',sc2_g1_time - sc1_g1_time,sc2_r1_time - sc1_r1_time,'']
]


time_df = pd.DataFrame(data_space, columns=['Scenario', 'Agent G1', 'Agent R1', 'Time Diff'])
theader = '<b>Time Complexity:</b>'+time_df.to_html(index=False)
HTML(theader) 
 

Scenario,Agent G1,Agent R1,Time Diff
SC1,17,14,-3
SC2,10,8,-2
DIFF,-7,-6,


### 6.	Provide your comparitive analysis or findings in no more than 3 lines in below section

<b>Comparison :</b> 
<li>Both the agents perform better in terms of time and space complexity in scenario 2 (grid 2)
<li>In scenario 1, the performance of the agents depends heavily on the start node of the agents. If G1 agent is placed above the red diagonal rooms in the centre, more number of rooms are opened to reach the node. However if the agent G1 is placed below the red diagonal rooms, it reaches the goal quickly
<li>For both the scenarios, agent R1, gives search path in terms of both time and cost efficiency
<li>In scenario 2, the checker board pattern negates the room penalty and the color penalty as it moves across rooms. As moving across rooms are negating the room penalties, the manhattan distance take prominance in determining the best path. This is evident from the evaluation results where we can see scenario 2 giving better time and cost efficiency. 
<br>________________________________________________________

_________________________________________________________